In [151]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
# from models.vince_models import Net2B

In [152]:
# load the dataset

train_input,train_target, train_classes, test_input, test_target, test_classes = load()
train_target=train_target.float() # necessary ?
test_target=test_target.float()

In [153]:
class Net2(nn.Module):

    def __init__(self, nb_hidden):
        super(Net2, self).__init__()
        
        # number of input channels is 2.
        self.conv1 = nn.Conv2d(2, 32, kernel_size = 5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 1) # single output
        self.sigmoid = nn.Sigmoid()  
        
    def forward(self,x):
    
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride = 1))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 3, stride = 3))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        x = self.sigmoid(x) 
        return x # returns a probability 

In [154]:
def train_binary(model, epoch, train_input, train_target, batch_size):  
    # train a binary classifier by minimizing the binary cross entropy loss of a sigmoid output
    criterion = nn.BCELoss() # not BCELossWithLogits because sigmoid activation of output already applied in model architecture
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    train_loss = 0
    model.train()
    for input_, target in (zip(train_input.split(batch_size), train_target.split(batch_size))):
        output = model(input_)
        optimizer.zero_grad()
        batch_loss = criterion(output, target.unsqueeze(1))
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss   
    print('Train Epoch: {}  | Loss {:.6f}'.format(
                epoch, train_loss.item()))
    

def test_binary(model, test_input, test_target, batch_size):
    # test a binary classifier using sigmoid followed by rounding ptobability to 0 or 1
    criterion = nn.BCELoss() 
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in zip(test_input.split(batch_size), test_target.split(batch_size)):
            output = model(data)
            batch_loss = criterion(output, target.unsqueeze(1))
            test_loss += batch_loss
            pred = output 
            correct += torch.round(output).squeeze().eq(target).sum() # only need to rounf because sigmoid already applied in model 
        print('\nTest set:  Loss: {:.4f}, Accuracy: {:.0f}%\n'.format(
        test_loss.item(), 100 * correct/len(test_target)))

In [157]:
###############################
###### Binary Classifier ######
###############################

# initialize
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)
model = Net2(100)

print(model)

# train
nb_epochs = 50
for epoch in range(1, nb_epochs):
    train_binary(model, epoch, train_input, train_target, 100)

Net2(
  (conv1): Conv2d(2, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [158]:
# test
test_binary(model, test_input, test_target, 100)